In [1]:
import numpy as np

### Creating your own GRN

In [13]:
num_genes = 15
cell_types = 3
master_regulators = 3


In [ ]:
def generate_transition_matrix(size):
    matrix = np.random.rand(size, size)
    matrix /= matrix.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return matrix



In [4]:
# from ortools.sat.python import cp_model

# def build_transition_matrix(cell_types, granularity=100):
#     model = cp_model.CpModel()
#     matrix = []

#     # Create variables for each transition
#     for i in range(cell_types):
#         row = []
#         for j in range(cell_types):
#             if i == j:
#                 # No self-transitions
#                 var = model.NewIntVar(0, 0, f"p_{i}_{j}")
#             else:
#                 var = model.NewIntVar(0, granularity, f"p_{i}_{j}")
#             row.append(var)
#         matrix.append(row)

#     # Create a boolean variable that represents whether a cell is the starting cell
#     start_cell_vars = []
#     for i in range(cell_types):
#         start_cell_vars.append(model.NewBoolVar(f"start_cell_{i}"))

#     # Add constraints to ensure that exactly one cell is the start cell
#     model.Add(sum(start_cell_vars) == 1)

#     # Ensure no transitions into the start cell (column for start cell should have all zeros)
#     for i in range(cell_types):
#         # For each cell, we must have no transitions from other cells into the start cell
#         model.Add(sum(matrix[j][i] for j in range(cell_types) if j != i) == 0).OnlyEnforceIf(start_cell_vars[i])

#     # Ensure the start cell has outgoing transitions (at least one non-zero in the row)
#     for i in range(cell_types):
#         model.Add(sum(matrix[i][j] for j in range(cell_types) if j != i) > 0).OnlyEnforceIf(start_cell_vars[i])

#     for i in range(cell_types):
#         # Whether cell i sends transitions (excluding self)
#         send_vars = [matrix[i][j] for i in range(cell_types) if j != i]
#         send_nonzero = model.NewBoolVar(f"send_{i}")
#         model.Add(sum(send_vars) > 0).OnlyEnforceIf(send_nonzero)
#         model.Add(sum(send_vars) == 0).OnlyEnforceIf(send_nonzero.Not())

#         # Whether cell i receives transitions (excluding self)
#         recv_vars = [matrix[j][i] for j in range(cell_types) if j != i]
#         recv_nonzero = model.NewBoolVar(f"recv_{i}")
#         model.Add(sum(recv_vars) > 0).OnlyEnforceIf(recv_nonzero)
#         model.Add(sum(recv_vars) == 0).OnlyEnforceIf(recv_nonzero.Not())

#         # Must either send or receive
#         model.AddBoolOr([send_nonzero, recv_nonzero])

#     # Solve
#     solver = cp_model.CpSolver()
#     status = solver.Solve(model)

#     if status in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
#         print("Transition Matrix:")
#         result = []
#         for i in range(cell_types):
#             row = []
#             for j in range(cell_types):
#                 val = solver.Value(matrix[i][j]) / granularity
#                 row.append(round(val, 2))
#             result.append(row)
#             print("\t".join(map(str, row)))
#         return result
#     else:
#         print("No solution found.")
#         return None

# # Example: 3 cell types
# build_transition_matrix(cell_types=3)
from ortools.sat.python import cp_model

def build_transition_matrix(cell_types, granularity=100):
    model = cp_model.CpModel()
    
    # Create a cell_types x cell_types matrix of integer variables.
    # Off-diagonal entries (i != j) are in [0, granularity] (representing probabilities scaled by granularity).
    # Diagonal entries (self-transitions) are fixed to 0.
    matrix = []
    for i in range(cell_types):
        row_vars = []
        for j in range(cell_types):
            if i == j:
                var = model.NewIntVar(0, 0, f"p_{i}_{j}")
            else:
                var = model.NewIntVar(0, granularity, f"p_{i}_{j}")
            row_vars.append(var)
        matrix.append(row_vars)
    
    # Create Boolean variables to mark start and terminal cells.
    start_cell = [model.NewBoolVar(f"start_cell_{i}") for i in range(cell_types)]
    terminal_cell = [model.NewBoolVar(f"terminal_cell_{i}") for i in range(cell_types)]
    
    # Exactly one start cell.
    model.Add(sum(start_cell) == 1)
    # Exactly one terminal cell.
    model.Add(sum(terminal_cell) == 1)
    
    # A cell cannot be both start and terminal.
    for i in range(cell_types):
        model.Add(start_cell[i] + terminal_cell[i] <= 1)
    
    # Create ordering variables for acyclicity.
    # They represent a permutation (0 to cell_types-1) and help prevent cycles.
    order = [model.NewIntVar(0, cell_types - 1, f"order_{i}") for i in range(cell_types)]
    model.AddAllDifferent(order)
    
    # For each cell, define its incoming and outgoing sums (excluding self-transitions).
    for i in range(cell_types):
        outgoing = sum(matrix[i][j] for j in range(cell_types) if j != i)
        incoming = sum(matrix[j][i] for j in range(cell_types) if j != i)
        
        # For a start cell: no incoming transitions and full outgoing probability.
        model.Add(incoming == 0).OnlyEnforceIf(start_cell[i])
        model.Add(outgoing == granularity).OnlyEnforceIf(start_cell[i])
        
        # For a terminal cell: no outgoing transitions and at least one incoming.
        model.Add(outgoing == 0).OnlyEnforceIf(terminal_cell[i])
        model.Add(incoming >= 1).OnlyEnforceIf(terminal_cell[i])
        
        # For any cell that is not terminal, force full outgoing probability.
        model.Add(outgoing == granularity).OnlyEnforceIf(terminal_cell[i].Not())
    
    # Prevent cycles:
    # For every possible transition i->j, if the transition is “active” (i.e. its value > 0),
    # then enforce that order[i] < order[j].
    for i in range(cell_types):
        for j in range(cell_types):
            if i == j:
                continue
            transition_active = model.NewBoolVar(f"transition_{i}_{j}_active")
            model.Add(matrix[i][j] >= 1).OnlyEnforceIf(transition_active)
            model.Add(matrix[i][j] == 0).OnlyEnforceIf(transition_active.Not())
            model.Add(order[i] < order[j]).OnlyEnforceIf(transition_active)
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
        result = []
        print("Transition Matrix:")
        for i in range(cell_types):
            row = []
            for j in range(cell_types):
                val = solver.Value(matrix[i][j]) / granularity  # Convert back to probability
                row.append(round(val, 2))
            result.append(row)
            print("\t".join(map(str, row)))
        print("Start cell:", [i for i in range(cell_types) if solver.Value(start_cell[i])])
        print("Terminal cell:", [i for i in range(cell_types) if solver.Value(terminal_cell[i])])
        print("Order:", [solver.Value(order[i]) for i in range(cell_types)])
        return result
    else:
        print("No solution found")
        return None

# Example usage: Build a 4x4 transition matrix.
build_transition_matrix(cell_types=5)



Transition Matrix:
0.0	0.01	0.01	0.97	0.01
0.0	0.0	0.0	0.0	1.0
0.0	1.0	0.0	0.0	0.0
0.0	0.0	0.0	0.0	1.0
0.0	0.0	0.0	0.0	0.0
Start cell: [0]
Terminal cell: [4]
Order: [0, 3, 2, 1, 4]


[[0.0, 0.01, 0.01, 0.97, 0.01],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]

In [81]:
import numpy as np

def build_transition_matrix(cell_types, granularity=100):
    # Initialize matrix of zeros
    matrix = np.zeros((cell_types, cell_types))
    
    # Randomly assign transitions (non-zero entries less than 1)
    for i in range(cell_types):
        for j in range(cell_types):
            if i != j:  # No self transitions
                matrix[i][j] = np.random.uniform(0, 1)  # Values between 0 and 1
    
    # Ensure that exactly one row is terminal (all zeros)
    terminal_row_idx = np.random.choice(cell_types)
    matrix[terminal_row_idx, :] = 0  # Set the entire row to zero
    
    # Ensure that other rows have at least one non-zero entry
    for i in range(cell_types):
        if i != terminal_row_idx:
            if np.sum(matrix[i, :]) == 0:  # Row has all zeros
                non_terminal_col = np.random.choice(cell_types)
                matrix[i, non_terminal_col] = np.random.uniform(0.01, 1)  # A small positive number
    
    # Ensure that for a terminal row, its corresponding column has non-zero entries
    for j in range(cell_types):
        if np.sum(matrix[:, j]) == 0:  # If the column j has all zeros
            non_terminal_row = np.random.choice([k for k in range(cell_types) if k != terminal_row_idx])
            matrix[non_terminal_row, j] = np.random.uniform(0.01, 1)  # A small positive number
    
    return matrix

# Example: 3 cell types
transition_matrix = build_transition_matrix(3)

# Display the matrix
print("Transition Matrix:")
print(transition_matrix)


Transition Matrix:
[[0.         0.67739934 0.97767144]
 [0.         0.         0.        ]
 [0.26539152 0.07749516 0.        ]]


No solution found
